# SQL Query on Netflix Dataset

Greetings, everyone! Within this Jupyter Notebook, my focus will be on examining the Netflix Dataset, which I've obtained from Kaggle (https://www.kaggle.com/datasets/shivamb/netflix-shows). 

I'll leverage Pandas and the SQLAlchemy library to to transform the CSV file into a DataFrame, as well as execute SQL queries on the dataset.

## Step 1: Install and Import Relevant Libraries and Packages

We will require pandas and SQLAlchemy, so I will be installing and importing them.

In [1]:
!pip install pandas sqlalchemy

In [2]:
import pandas as pd
from sqlalchemy import create_engine

## Step 2: Read CSV File

The dataset that I have taken from Kaggle is in a .csv format. I will be using Pandas to read the dataset from my local computer as a DataFrame.

In [33]:
file_path = 'netflix_titles.csv'
df = pd.read_csv(file_path)

In [34]:
df = df.dropna()

In [35]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...
12,s13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic","September 23, 2021",2021,TV-MA,127 min,"Dramas, International Movies",After most of her family is murdered in a terr...
24,s25,Movie,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,"September 21, 2021",1998,TV-14,166 min,"Comedies, International Movies, Romantic Movies",When the father of the man she loves insists t...


## Step 3: Creat In-Memory Database and Save DataFrame

Next, I will be creating an SQLite database engine in memory. The sqlite:///:memory: URL indicates that the database will exist only in memory and will not be persisted to disk. This is useful for temporary storage and testing.

Subsequently, I will write the contents of the DataFrame (df) into an SQLite table named 'netflix_titles' within the in-memory database specified by the engine. The index=False parameter indicates that the DataFrame's index should not be included as a separate column in the database table.

In [36]:
engine = create_engine('sqlite:///:memory:')  # Use an in-memory SQLite database
df.to_sql('netflix_titles', engine, index=False)

5332

## Step 4: Basic Queries

Next, I will be running basic queries to:
- Get information about the dataset
- Display the types of netflix titles
- Display the number of shows added by country

In [45]:
query = "SELECT * FROM netflix_titles LIMIT 5"
result = pd.read_sql_query(query, engine)
print(result)

  show_id     type                          title             director  \
0      s8    Movie                        Sankofa         Haile Gerima   
1      s9  TV Show  The Great British Baking Show      Andy Devonshire   
2     s10    Movie                   The Starling       Theodore Melfi   
3     s13    Movie                   Je Suis Karl  Christian Schwochow   
4     s25    Movie                          Jeans           S. Shankar   

                                                cast  \
0  Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...   
1  Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...   
2  Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...   
3  Luna Wedler, Jannis Niewöhner, Milan Peschel, ...   
4  Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...   

                                             country          date_added  \
0  United States, Ghana, Burkina Faso, United Kin...  September 24, 2021   
1                                     United Kingdom  September 24

In [38]:
query = "SELECT COUNT(*) AS total_rows, MIN(release_year) AS min_year, MAX(release_year) AS max_year FROM netflix_titles"
result = pd.read_sql_query(query, engine)
print(result)

   total_rows  min_year  max_year
0        5332      1942      2021


In [39]:
query = "SELECT DISTINCT type FROM netflix_titles"
result = pd.read_sql_query(query, engine)
print(result)

      type
0    Movie
1  TV Show


In [40]:
query = "SELECT country, COUNT(*) AS show_count FROM netflix_titles GROUP BY country ORDER BY show_count DESC"
result = pd.read_sql_query(query, engine)
print(result)

                                               country  show_count
0                                        United States        1846
1                                                India         875
2                                       United Kingdom         183
3                                               Canada         107
4                                                Spain          91
..                                                 ...         ...
599   Argentina, France, United States, Germany, Qatar           1
600                                  Argentina, France           1
601                             Argentina, Chile, Peru           1
602  Argentina, Brazil, France, Poland, Germany, De...           1
603                                  , France, Algeria           1

[604 rows x 2 columns]


## Step 5: Additional Queries

I will be running additional queries to:
- Find the Netflix titles that were released in 2021.
- Find the Netflix titles that have "Comedies" in their tag.
- Find the Netflix titles which were directed by Mike Flanagan
- Find the director which has the most shows in this dataset.

In [41]:
query = "SELECT title, release_year FROM netflix_titles WHERE type = 'Movie' AND release_year = 2021"
result = pd.read_sql_query(query, engine)
print(result)

                             title  release_year
0                     The Starling          2021
1                     Je Suis Karl          2021
2                             Kate          2021
3                       Thimmarusu          2021
4                    The Water Man          2021
..                             ...           ...
141  Tribhanga - Tedhi Medhi Crazy          2021
142    What Would Sophia Loren Do?          2021
143                       Wish You          2021
144                       Charming          2021
145      What Happened to Mr. Cha?          2021

[146 rows x 2 columns]


In [42]:
query = "SELECT title, listed_in FROM netflix_titles WHERE listed_in LIKE '%Comedies%';"
result = pd.read_sql_query(query, engine)
print(result)

                     title                                        listed_in
0             The Starling                                 Comedies, Dramas
1                    Jeans  Comedies, International Movies, Romantic Movies
2                Grown Ups                                         Comedies
3     Omo Ghetto: the Saga             Action & Adventure, Comedies, Dramas
4                Show Dogs               Children & Family Movies, Comedies
...                    ...                                              ...
1578           Young Adult             Comedies, Dramas, Independent Movies
1579  Yours, Mine and Ours               Children & Family Movies, Comedies
1580              Zed Plus           Comedies, Dramas, International Movies
1581            Zombieland                          Comedies, Horror Movies
1582                  Zoom               Children & Family Movies, Comedies

[1583 rows x 2 columns]


In [43]:
query = "SELECT title, director FROM netflix_titles WHERE director LIKE '%Mike Flanagan%'"
result = pd.read_sql_query(query, engine)
print(result)

           title       director
0  Before I Wake  Mike Flanagan
1  Gerald's Game  Mike Flanagan
2           Hush  Mike Flanagan


In [44]:
query = "SELECT director, COUNT(*) AS show_count FROM netflix_titles GROUP BY director ORDER BY show_count DESC LIMIT 5"
result = pd.read_sql_query(query, engine)
print(result)

                 director  show_count
0  Raúl Campos, Jan Suter          18
1            Marcus Raboy          15
2               Jay Karas          14
3     Cathy Garcia-Molina          13
4         Youssef Chahine          12
